In [13]:
import sys
import logging

import http.client
import json

import pandas as pd

In [14]:
logger = logging.getLogger(__name__)
# logger.setLevel(logging.INFO)
logger.setLevel(logging.DEBUG)

c_handler = logging.StreamHandler(sys.stdout)
c_format = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
# c_format = logging.Formatter(f'%(levelname)-8s: %(message)s')
c_handler.setFormatter(c_format)

for handler in logger.handlers:
    logger.removeHandler(handler)
logger.addHandler(c_handler)

In [15]:
logger.debug('meow')
logger.info('aku adalah anak woof woof')

2020-03-11 06:36:27,107 - __main__ - DEBUG - meow
2020-03-11 06:36:27,111 - __main__ - INFO - aku adalah anak woof woof


In [39]:
TYPE = 'dev'

SQUAD_DATASET_PATH = f'../data/interim/{TYPE}-v2.0-translated_fixed.json'

df_squad = pd.read_json(SQUAD_DATASET_PATH)
df_squad = df_squad
print(df_squad.shape)
print(df_squad)

(35, 2)
                                            title  \
0                                       Normandia   
1                    Teori kompleksitas komputasi   
2                              California Selatan   
3                          Sky (Kerajaan Inggris)   
4                            Victoria (Australia)   
5                                        Huguenot   
6                                       Mesin uap   
7                                         Oksigen   
8                              Krisis minyak 1973   
9                                 Hukum Uni Eropa   
10                             hutan hujan Amazon   
11                                     Ctenophora   
12                             Fresno, California   
13                               Pergantian paket   
14                                 Kematian kelam   
15                                        Geologi   
16                                        Farmasi   
17                            Pembangk

In [42]:
def get_pos_tag(text):
    conn = http.client.HTTPConnection("10.181.131.244:5500")
    payload = json.dumps({"text": text})
    headers = {
        'content-type': "application/json",
        'x-api-key': ""
        }

    conn.request("POST", "/", payload, headers)
    res = conn.getresponse()
    data = json.loads(res.read().decode("utf-8"))

    return data

def get_ner(text):
    conn = http.client.HTTPConnection("10.181.131.244:10009")
    payload = json.dumps({"text": text})
    headers = {
        'content-type': "application/json",
        'x-api-key': ""
        }

    conn.request("POST", "/", payload, headers)
    res = conn.getresponse()
    data = json.loads(res.read().decode("utf-8"))

    return data

text = "Frédéric Chopin! adalah, (1907–1986) (1907-1986) anak-anak (children): gembala. Andi's sheep is an '03 R&B player; 奉獻 km² Jīdū °5 вера ʰp. Totalnya 10.000 rupiah u?yea u!yea u,yea u.yea kura-kura bolak-balik"
pos_tag = get_pos_tag(text)
ner = get_ner(text)

In [43]:
print(pos_tag)
print(ner)

{'postags': [[['Frederic', 'NNP'], ['Chopin', 'NNP'], ['!', 'PUN']], [['adalah', 'VBL'], [',', 'PUN'], ['(', 'PUN'], ['1907', 'NUM'], ['-', 'PUN'], ['1986', 'NUM'], [')', 'PUN'], ['(', 'PUN'], ['1907', 'NUM'], ['-', 'PUN'], ['1986', 'NUM'], [')', 'PUN'], ['anak-anak', 'NNO'], ['(', 'PUN'], ['children', 'NNO'], [')', 'PUN'], [':', 'PUN'], ['gembala', 'NNO'], ['.', 'PUN']], [['Andi', 'NNP'], ["'", 'PUN'], ['s', 'NNP'], ['sheep', 'NNP'], ['is', 'PPO'], ['an', 'PUN'], ["'", 'PUN'], ['03', 'NUM'], ['R', 'NNP'], ['&', 'PUN'], ['B', 'NNP'], ['player', 'NNP'], [';', 'PUN'], ['Feng', 'NNP'], ['Xian', 'NNP'], ['km2', 'NNP'], ['Jidu', 'NNP'], ['deg5', 'NNP'], ['vera', 'NNP'], ['kp', 'NNP'], ['.', 'PUN']], [['Total', 'NNO'], ['nya', 'PRK'], ['10.000', 'NUM'], ['rupiah', 'NNO'], ['u', 'ADJ'], ['?', 'PUN']], [['yea', 'NUM'], ['u', 'NNP'], ['!', 'PUN']], [['yea', 'NNO'], ['u', 'PUN'], [',', 'PUN'], ['yea', 'NNP'], ['u.ye', 'NNP'], ['a', 'PPO'], ['kura-kura', 'NNO'], ['bolak-balik', 'VBI']]]}
{'entiti

In [ ]:
SLEEP_TIME = 10

def save_wait_recall_api(payload, call_api_func, df, path):
    print(f'Problem when calling API.. Saving, then sleeping for {SLEEP_TIME} seconds...')
    df.to_json(path)
    time.sleep(SLEEP_TIME)
    return call_api_func(payload)

In [ ]:
class LastIdxTracker():
    def __init__(self, last_topic_idx, last_content_idx):
        self.topic = last_topic_idx
        self.content = last_content_idx

In [ ]:
def add_postag_and_ner(taken_topic_idx, taken_context_idx, path, last_idx_tracker, logger=logger):
    qas = df_squad.iloc[taken_topic_idx]['paragraphs'][taken_context_idx]['qas']
    context = df_squad.iloc[taken_topic_idx]['paragraphs'][taken_context_idx]['context']
#     logger.debug(f'{context}\n')

    try:
        context_postags = get_pos_tag(context)
    except:
        context_postags = save_wait_recall_api(context, get_pos_tag, df_squad, path)
                    
    try:
        context_ner = get_ner(context)
    except:
        context_ner = save_wait_recall_api(context, get_ner, df_squad, path)
        
    df_squad.iloc[taken_topic_idx]['paragraphs'][taken_context_idx].update(context_postags)
    df_squad.iloc[taken_topic_idx]['paragraphs'][taken_context_idx].update(context_ner)
    
    last_idx_tracker.topic = taken_topic_idx
    last_idx_tracker.content = taken_context_idx
    
    for question in qas:
        try:
            question_postags = get_pos_tag(question['question'])
        except:
            question_postags = save_wait_recall_api(question['question'], get_pos_tag, df_squad, path)

        try:
            question_ner = get_ner(question['question'])
        except:
            question_ner = save_wait_recall_api(question['question'], get_ner, df_squad, path)
        question.update(question_postags)
        question.update(question_ner)

In [45]:
last_idx_tracker = LastIdxTracker(0, 0)

In [44]:
print(last_idx_tracker.topic)
print(last_idx_tracker.content)

441
0


In [48]:
import time


SAVE_PATH = f'../data/interim/{TYPE}-v2.0-translated_fixed_enhanced.json'

start_time = time.time()
for taken_topic_idx in range(last_idx_tracker.topic, df_squad.shape[0]):
    logger.debug(f'Topic: {taken_topic_idx}')
    for taken_context_idx in range(last_idx_tracker.content, len(df_squad.iloc[taken_topic_idx]['paragraphs'])):
        logger.debug(f'\t{taken_context_idx}')
        add_postag_and_ner(taken_topic_idx, taken_context_idx, SAVE_PATH, last_idx_tracker)
#         logger.debug(df_squad.iloc[taken_topic_idx]['paragraphs'][taken_context_idx])
    last_idx_tracker.content = 0
#     if last_idx_tracker.topic==143: break
print(time.time() - start_time)

2020-03-11 21:44:06,172 - __main__ - DEBUG - Topic: 0
2020-03-11 21:44:06,175 - __main__ - DEBUG - 	0
2020-03-11 21:44:08,683 - __main__ - DEBUG - 	1
2020-03-11 21:44:10,513 - __main__ - DEBUG - 	2
2020-03-11 21:44:11,473 - __main__ - DEBUG - 	3
2020-03-11 21:44:13,567 - __main__ - DEBUG - 	4
2020-03-11 21:44:14,323 - __main__ - DEBUG - 	5
2020-03-11 21:44:15,393 - __main__ - DEBUG - 	6
2020-03-11 21:44:17,365 - __main__ - DEBUG - 	7
2020-03-11 21:44:18,458 - __main__ - DEBUG - 	8
2020-03-11 21:44:21,277 - __main__ - DEBUG - 	9
2020-03-11 21:44:22,883 - __main__ - DEBUG - 	10
2020-03-11 21:44:24,373 - __main__ - DEBUG - 	11
2020-03-11 21:44:26,122 - __main__ - DEBUG - 	12
2020-03-11 21:44:27,877 - __main__ - DEBUG - 	13
2020-03-11 21:44:29,444 - __main__ - DEBUG - 	14
2020-03-11 21:44:30,884 - __main__ - DEBUG - 	15
2020-03-11 21:44:32,131 - __main__ - DEBUG - 	16
2020-03-11 21:44:33,603 - __main__ - DEBUG - 	17
2020-03-11 21:44:34,569 - __main__ - DEBUG - 	18
2020-03-11 21:44:36,043 -

In [ ]:
df_squad.iloc[taken_topic_idx]['paragraphs'][1]

In [ ]:
# df_squad = df_squad[df_squad.index < 9]
# df_squad.iloc[-1]['paragraphs'][-1]

In [49]:
print(f'Saving to: {SAVE_PATH}')
df_squad.to_json(SAVE_PATH)

Saving to: Datasets/SQuAD/v2.0/dev-v2.0-translated_fixed_enhanced.json
